In [1]:
import sys
sys.path.append('..') 
from ridge_utils.DataSequence import DataSequence
import torch
import pickle
import os
from transformers import BertTokenizerFast
from torch.utils.data import DataLoader
from encoder import Encoder
from data import TextDataset
from train_encoder import train_bert

# Train Encoder

In [ ]:
# Load the podcast text data
data_path = "/ocean/projects/mth240012p/shared/data"
raw_text_path = os.path.join(data_path, "raw_text.pkl")

with open(raw_text_path, "rb") as f:
    raw_text = pickle.load(f)

print(f"Loaded {len(raw_text)} podcast stories")

# Flatten all the words from all stories into a single list
all_sentences = []
for story in raw_text.values():
    all_sentences.extend(story.data)

# Tokenizer & Dataset
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
dataset = TextDataset(all_sentences, tokenizer, max_len=32)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Create encoder model
device = "cuda" if torch.cuda.is_available() else "cpu"
# encoder = Encoder(vocab_size=tokenizer.vocab_size)
# encoder = encoder.to(device)

import torch
import itertools
import matplotlib.pyplot as plt

# Fixed settings
lr = 3e-4
epochs = 15
batch_size = 32
num_heads = 4
intermediate_size = 512

# Things to tune
hidden_sizes = [128, 256, 512]
num_layers_list = [2, 4, 6]

# Store results
results = {}

# Loop over all combinations
for hidden_size, num_layers in itertools.product(hidden_sizes, num_layers_list):
    print(f"Training model with hidden_size={hidden_size}, num_layers={num_layers}")

    # Initialize new encoder
    encoder = Encoder(
        vocab_size=tokenizer.vocab_size,
        hidden_size=hidden_size,
        num_heads=num_heads,
        num_layers=num_layers,
        intermediate_size=intermediate_size,
        max_len=512
    )
    encoder = encoder.to(device)

    # Train and get losses
    train_losses, val_losses = train_bert(
        model=encoder,
        dataloader=dataloader,
        tokenizer=tokenizer,
        epochs=epochs,
        lr=lr,
        device=device
    )

    # Save
    results[(hidden_size, num_layers)] = {
        'train_losses': train_losses,
        'val_losses': val_losses
    }

print("Finished hyperparameter tuning!")


# Plot validation loss curves for all runs
plt.figure(figsize=(10, 6))

for (hidden_size, num_layers), losses in results.items():
    val_losses = losses['val_losses']
    label = f"hs={hidden_size}, nl={num_layers}"
    plt.plot(range(1, len(val_losses)+1), val_losses, marker='o', label=label)

plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.title('Validation Loss for Different Hyperparameters')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Loaded 109 podcast stories
Training model with hidden_size=128, num_layers=2
Epoch [1/15], Train Loss: 2.3667, Val Loss: 2.2113
Epoch [2/15], Train Loss: 2.1728, Val Loss: 2.1585
Epoch [3/15], Train Loss: 2.1292, Val Loss: 2.2192
Epoch [4/15], Train Loss: 2.1172, Val Loss: 2.1168
Epoch [5/15], Train Loss: 2.1089, Val Loss: 2.1219
Epoch [6/15], Train Loss: 2.0830, Val Loss: 2.0350
Epoch [7/15], Train Loss: 2.0747, Val Loss: 2.0338
Epoch [8/15], Train Loss: 2.0671, Val Loss: 2.1197
Epoch [9/15], Train Loss: 2.0488, Val Loss: 2.0863
Epoch [10/15], Train Loss: 2.0568, Val Loss: 2.1351
Epoch [11/15], Train Loss: 2.0822, Val Loss: 2.0783
Epoch [12/15], Train Loss: 2.0796, Val Loss: 2.1745
Epoch [13/15], Train Loss: 2.0771, Val Loss: 2.1639
Epoch [14/15], Train Loss: 2.0895, Val Loss: 2.1893
Epoch [15/15], Train Loss: 2.1082, Val Loss: 2.1990
Training model with hidden_size=128, num_layers=4
Epoch [1/15], Train Loss: 2.3595, Val Loss: 2.1906
Epoch [2/15], Train Loss: 2.1851, Val Loss: 2.1514


In [ ]:
# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Build encoder with best hyperparameters
encoder = Encoder(
    vocab_size=tokenizer.vocab_size,
    hidden_size=128,     # BEST hidden size
    num_heads=4,         # same as before
    num_layers=2,        # BEST number of layers
    intermediate_size=512,
    max_len=512
)

encoder = encoder.to(device)

# Prepare your dataset (same as before)
dataset = TextDataset(all_sentences, tokenizer, max_len=32)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Train
print("Training BEST encoder")
train_losses, val_losses = train_bert(
    model=encoder,
    dataloader=dataloader,
    tokenizer=tokenizer,
    epochs=5,
    lr=5e-4,
    device=device
)

# Save
torch.save(encoder.state_dict(), "pretrained_encoder_best.pt")
print("Saved best encoder to pretrained_encoder_best.pt")


# Generate All Embeddings 

In [ ]:
import torch
import numpy as np
import pickle, os
from transformers import BertTokenizerFast
from encoder import Encoder
from preprocessing import downsample_word_vectors, make_delayed

# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
# tokenizer = BertTokenizerFast.from_pretrained("/jet/home/cpestell/bert-base-uncased")  # your local copy
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

encoder = Encoder(
    vocab_size=tokenizer.vocab_size,
    hidden_size=128,  # best hyperparameter you found!
    num_heads=4,      # same as you trained
    num_layers=2,     # best hyperparameter you found!
    intermediate_size=512,  # same as training
    max_len=512
)

encoder.load_state_dict(torch.load("pretrained_encoder_best.pt", map_location=device))
encoder.to(device)
encoder.eval()

# Load raw text
data_path = "/ocean/projects/mth240012p/shared/data"
with open(os.path.join(data_path, "raw_text.pkl"), "rb") as f:
    raw_text = pickle.load(f)

story_names = list(raw_text.keys())

# Step 1: Generate encoder embeddings
encoder_vectors = {}
for story in story_names:
    words = raw_text[story].data
    inputs = tokenizer(words, padding=True, truncation=True, return_tensors="pt", max_length=64)

    input_ids = inputs["input_ids"].to(device)
    token_type_ids = inputs["token_type_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        hidden = encoder(input_ids, token_type_ids, attention_mask)

    # Mean over tokens (seq_len)
    sentence_embeddings = hidden.mean(dim=1).cpu().numpy()
    encoder_vectors[story] = sentence_embeddings
    # print(f"{story}: Encoder shape = {encoder_vectors[story].shape}")

# Step 2: Downsample
wordseqs = raw_text
downsampled_encoder = downsample_word_vectors(story_names, encoder_vectors, wordseqs)

# Step 3: Trim
X_encoder_trimmed = {}
for story in downsampled_encoder:
    X_encoder_trimmed[story] = downsampled_encoder[story][5:-10, :]
    # print(f"Trimmed shape for {story}:", X_encoder_trimmed[story].shape)

# Step 4: Create lagged features
X_encoder_lagged = {}
for story in X_encoder_trimmed:
    X_encoder_lagged[story] = make_delayed(X_encoder_trimmed[story], delays=range(1, 5))
    # print(f"Lagged shape for {story}:", X_encoder_lagged[story].shape)


# Modelling and Evaluation

In [ ]:
import numpy as np
import os
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

# --------- Setup ---------
story_names = list(raw_text.keys())
train_stories, test_stories = train_test_split(story_names, test_size=0.2, random_state=42)

subject2_path = os.path.join(data_path, "subject2")
X_train = []
Y_train = []



for story in train_stories:
    if story in X_encoder_lagged:
        bold_path = os.path.join(subject2_path, f"{story}.npy")
        if os.path.exists(bold_path):
            Y = np.load(bold_path)
            X = X_encoder_lagged[story]
            min_len = min(X.shape[0], Y.shape[0])
            X_train.append(X[:min_len])
            Y_train.append(Y[:min_len])
        else:
            print(f"Missing Y file for: {story}")

X_train = np.vstack(X_train)
Y_train = np.vstack(Y_train)
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)

# --------- Clean NaNs ---------
mask = ~np.isnan(Y_train).any(axis=1)
X_train_clean = X_train[mask]
Y_train_clean = Y_train[mask]

# --------- Cross Validation ---------
sample_size = int(0.1 * X_train_clean.shape[0])  # 20% of the data
X_train_sub = X_train_clean[:sample_size]
Y_train_sub = Y_train_small[:sample_size]
alphas = np.logspace(-4, 4, 100)  # Example range for alpha
ridge = Ridge()
grid_search = GridSearchCV(ridge, param_grid={'alpha': alphas}, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_sub, Y_train_sub)
best_alpha = grid_search.best_params_['alpha']
print(f"\nBest alpha found: {best_alpha}")

# --------- Train Model ---------
ridge_best = Ridge(alpha=best_alpha)
ridge_best.fit(X_train_clean, Y_train_small)

# --------- Evaluate on Test Set ---------
X_test = []
Y_test = []

for story in test_stories:
    if story in X_encoder_lagged:
        bold_path = os.path.join(subject2_path, f"{story}.npy")
        if os.path.exists(bold_path):
            Y = np.load(bold_path)
            X = X_encoder_lagged[story]
            min_len = min(X.shape[0], Y.shape[0])
            X_test.append(X[:min_len])
            Y_test.append(Y[:min_len])
        else:
            print(f"Missing Y file for: {story}")

X_test = np.vstack(X_test)
Y_test = np.vstack(Y_test)

# Clean NaNs from test
mask = ~np.isnan(Y_test).any(axis=1)
X_test_clean = X_test[mask]
Y_test_clean = Y_test[mask]

# Predict
Y_pred = ridge_best.predict(X_test_clean)

# --------- Compute Correlation Coefficients (CCs) ---------
def compute_voxel_ccs(y_true, y_pred):
    ccs = []
    for i in range(y_true.shape[1]):
        try:
            cc, _ = pearsonr(y_true[:, i], y_pred[:, i])
        except:
            cc = np.nan
        ccs.append(cc)
    return np.array(ccs)

ccs_encoder = compute_voxel_ccs(Y_test_clean, Y_pred)

print(f"Mean CC: {np.mean(ccs_encoder):.4f}")
print(f"Median CC: {np.median(ccs_encoder):.4f}")
print(f"Top 1% CC: {np.percentile(ccs_encoder, 99):.4f}")
print(f"Top 5% CC: {np.percentile(ccs_encoder, 95):.4f}")

# --------- Visualization ---------
sns.set(style="whitegrid", font_scale=1.2)
plt.figure(figsize=(12, 6))

sns.histplot(ccs_encoder[~np.isnan(ccs_encoder)], bins=100, kde=True, color="#1f77b4", edgecolor="black", stat="count")
plt.axvline(np.mean(ccs_encoder), color='red', linestyle='--', label=f"Mean: {np.mean(ccs_encoder):.3f}")
plt.axvline(np.median(ccs_encoder), color='green', linestyle='-.', label=f"Median: {np.median(ccs_encoder):.3f}")
plt.title("Encoder Correlation Coefficient (CC) Distribution (5000 Voxels)", fontsize=16, weight="bold")
plt.xlabel("Correlation Coefficient", fontsize=13)
plt.ylabel("Number of Voxels", fontsize=13)
plt.legend()
plt.tight_layout()

# Save plot
save_dir = os.path.join("..", "results")
os.makedirs(save_dir, exist_ok=True)
plot_path = os.path.join(save_dir, "cc_distribution_encoder_subject2_5000vox.png")
plt.savefig(plot_path, dpi=300)
plt.show()

print(f"Plot saved to: {plot_path}")
